In [62]:
# Imports 
import numpy as np
from tqdm import trange,tqdm
import matplotlib.pyplot as plt

In [ ]:
# Constantes 
GOAL_SCORE = 100.0
ALPHA = 0.1
GAMMA = 0.95
EPSILON = 0.8
EPOCHS = 10000

# Variables
grid_rows = 3
grid_cols = 4

q_table = np.zeros((grid_rows, grid_cols, 4)) 

ACTIONS = ['up', 'down', 'left', 'right']

In [70]:
# Funciones del Entorno


def is_terminal_state(row, col, grid_rewards):
    """Verifica si el estado actual es terminal (meta)."""
    if grid_rewards[row, col] == GOAL_SCORE:
        return True
    return False


def get_next_action(row, col, q_table, epsilon, drunkenness=0.0):
    """
    Selecciona la siguiente acción.
    Prioridad:
    1. Exploración (epsilon-greedy).
    2. Explotación (mejor valor Q).
    o
    2. Drunkenness (movimiento aleatorio involuntario).
    """
    # Intención 
    if np.random.random() < epsilon:
        intended_action = np.random.randint(len(ACTIONS))
    else:
        intended_action = int(np.argmax(q_table[row, col]))

    # Drunkenness: falla al ejecutar la intención por lo que toma otra acción distinta
    if drunkenness > 0.0 and np.random.random() < drunkenness:
        other_actions = [i for i in range(len(ACTIONS)) if i != intended_action]
        return int(np.random.choice(other_actions))

    # Ejecutar intención
    return intended_action


def get_next_state(row, col, action_index, obstacles):
    """
    Calcula el nuevo estado (row, col) dado el estado actual y una ACCIÓN específica.
    Ya no decide la acción, solo ejecuta la física.
    """
    action = ACTIONS[action_index]
    new_row, new_col = row, col

    if action == "up" and row > 0:
        new_row -= 1
    elif action == "down" and row < grid_rows - 1:
        new_row += 1
    elif action == "left" and col > 0:
        new_col -= 1
    elif action == "right" and col < grid_cols - 1:
        new_col += 1

    # Choque con obstáculos
    if [new_row, new_col] in obstacles:
        return row, col

    return new_row, new_col


def get_shortest_path(
    start_row, start_col, q_table, grid_rewards, obstacles, drunkenness=0.0
):
    """
    Obtiene el camino más corto aprendida por la Q-Table desde un inicio hasta la meta.
    Usa epsilon=0 para seguir la política óptima.
    """
    if is_terminal_state(start_row, start_col, grid_rewards):
        return []

    current_row, current_col = start_row, start_col
    path = [[current_row, current_col]]

    while not is_terminal_state(current_row, current_col, grid_rewards):
        # epsilon=0.0 para explotar la mejor ruta
        action_index = get_next_action(
            current_row, current_col, q_table, epsilon=0.0, drunkenness=0.0
        )
        current_row, current_col = get_next_state(
            current_row, current_col, action_index, obstacles
        )
        path.append([current_row, current_col])

    return path

import pandas as pd
from IPython.display import display

def display_q_table_snapshots(snapshots):
    """
    Visualiza una lista de snapshots de Q-Table (para el Ejercicio 1 - Array 3D).
    Convierte cada snapshot (3, 4, 4) a un DataFrame 2D legible.
    """
    actions = ['up', 'down', 'left', 'right']
    
    for i, q_table in enumerate(snapshots):
        print(f"\n{'='*20} SNAPSHOT {i} {'='*20}")
        
        # Q-Table shape es (rows, cols, actions) -> (3, 4, 4)
        rows, cols, n_actions = q_table.shape
        
        data = []
        for r in range(rows):
            for c in range(cols):
                # Obtenemos los 4 valores Q para esta celda
                q_values = q_table[r, c]
                
                # Creamos una fila con: coordenadas + valores de cada acción
                row_data = {"Position": f"({r}, {c})"}
                for action_idx, action_name in enumerate(actions):
                    row_data[action_name] = q_values[action_idx]
                
                data.append(row_data)
        
        # Crear DataFrame
        df = pd.DataFrame(data)
        
        # Configurar 'Position' como índice para que se vea más limpio
        df.set_index("Position", inplace=True)
        
        display(df)
    

In [77]:
def train(q_table, grid_rewards, obstacles, 
          alpha=ALPHA, gamma=GAMMA, 
          epsilon=EPSILON, epsilon_min=0.01, epsilon_decay=0.995,
          conv_threshold=0.001, patience=10,
          drunkenness=0.0):
    """
    Entrena el agente usando Q-Learning.
    
    Args:
        q_table: Tabla Q inicial.
        grid_rewards: Matriz de recompensas del entorno.
        obstacles: Lista de coordenadas de obstáculos.
        alpha, gamma, epsilon: Hiperparámetros de Q-Learning.
        drunkenness: Probabilidad de que el agente se mueva aleatoriamente (ruido).
    """
    
    # Snapshots para análisis
    snapshots = [q_table.copy()]
    reward_history = []
    
    epoch_limit = 30000
    total_epochs = 0
    stable_epochs = 0
    is_converged = False
    
    if drunkenness > 0.0:
        print(f"Iniciando entrenamiento (Drunkenness={drunkenness})...")
    else:
        print("Iniciando entrenamiento...")

    for epoch in trange(epoch_limit):
        total_epochs += 1
        
        # Estado inicial fijo en (2,0)
        current_row, current_col = 2, 0 
        
        total_reward = 0
        old_q_table = q_table.copy()
        
        while not is_terminal_state(current_row, current_col, grid_rewards):
            # Guardamos estado anterior para actualizar Q(s,a)
            old_row, old_col = current_row, current_col
            
            # 1. Elegir Acción (Política + Drunkenness (si hubiere))
            action_index = get_next_action(
                current_row, current_col, 
                q_table, epsilon, drunkenness
            )
            
            # 2. Ejecutar Movimiento
            current_row, current_col = get_next_state(
                current_row, current_col, 
                action_index, obstacles
            )
            
            # 3. Obtener Recompensa
            reward = grid_rewards[current_row, current_col]
            
            # 4. Actualizar Q-Value 
            # Q(s,a) = Q(s,a) + alpha * (R + gamma * max(Q(s',a')) - Q(s,a))
            
            old_q_value = q_table[old_row, old_col, action_index]
            best_next_q = np.max(q_table[current_row, current_col])
            
            temporal_difference = reward + (gamma * best_next_q) - old_q_value
            new_q_value = old_q_value + (alpha * temporal_difference)
            
            q_table[old_row, old_col, action_index] = new_q_value
            
            total_reward += reward
            
        # Fin del epoch
        
        # Chequeo de Convergencia
        delta = np.linalg.norm(q_table - old_q_table)
        reward_history.append(total_reward)
        
        # Decay Epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        
        # Snapshots periódicos
        snapshots.append(q_table.copy())
        
        if delta < conv_threshold:
            stable_epochs += 1
            if stable_epochs >= patience:
                print(f"Converged after {total_epochs} epochs.")
                is_converged = True
                snapshots.append(q_table.copy()) # Guardar el convergido
                break
        else:
            stable_epochs = 0
            
    # Asegurar snapshot final si no está
    if len(snapshots) == 0 or not np.array_equal(q_table, snapshots[-1]):
        snapshots.append(q_table.copy())
        
    # Selección de snapshots representativos para return
    # (Inicio, 1/3, 2/3, Final) - aproximado
    n_snaps = len(snapshots)
    indices = [0, int(n_snaps*0.33), int(n_snaps*0.66), n_snaps-1]
    # Filtrar índices válidos y únicos (por si n_snaps es pequeño)
    indices = sorted(list(set([i for i in indices if i < n_snaps])))
    snapshots_output = [snapshots[i] for i in indices]
    
    # Métricas finales
    avg_reward_final = np.mean(reward_history[-100:]) if len(reward_history) >= 100 else np.mean(reward_history)
    avg_reward_total = np.mean(reward_history)
    
    return {
        "snapshots": snapshots_output,
        "avg_reward_final": avg_reward_final,
        "avg_reward_total": avg_reward_total,
        "total_epochs": total_epochs,
        "is_converged": is_converged,
        "reward_history": reward_history
    }

In [82]:
# ==========================================
# Ejecución Ejercicio 1.a (Entorno basico)
# ==========================================

rewards = np.full((grid_rows, grid_cols), -1)
rewards[0, 3] = GOAL_SCORE # meta
obstacles = [[1,1]] # obstaculo
rewards[1,1] = -100

print("Recompensas 1.a:")
print(rewards)

# 1. Reiniciamos la Q-Table para asegurar entrenamiento desde cero
q_table = np.zeros((grid_rows, grid_cols, 4)) 

# 2. Entrenamos
print("\nEntrenando Agente 1.a...")
results_1a = train(
    q_table=q_table, 
    grid_rewards=rewards, 
    obstacles=obstacles, 
    alpha=ALPHA,
    gamma=GAMMA,
    epsilon=EPSILON,
    conv_threshold=0.0001, 
    patience=20,
    drunkenness=0.0 # Agente sobrio
)

# 3. Resultados
print(f"\nEntrenamiento finalizado en {results_1a['total_epochs']} épocas.")
print(f"¿Convergió?: {results_1a['is_converged']}")
print(f"Recompensa Promedio Final: {results_1a['avg_reward_final']:.2f}")

# 4. Verificación del Camino Aprendido
print("\nCamino Óptimo Aprendido (desde [2,0]):")
path_1a = get_shortest_path(2, 0, q_table, rewards, obstacles)
print(path_1a)

# 5. Visualización de las Q-Tables
snapshots_1a = results_1a['snapshots']
display_q_table_snapshots(snapshots_1a)

Recompensas 1.a:
[[  -1   -1   -1  100]
 [  -1 -100   -1   -1]
 [  -1   -1   -1   -1]]

Entrenando Agente 1.a...
Iniciando entrenamiento...


  2%|▏         | 530/30000 [00:00<00:06, 4602.43it/s]

Converged after 531 epochs.

Entrenamiento finalizado en 531 épocas.
¿Convergió?: True
Recompensa Promedio Final: 95.62

Camino Óptimo Aprendido (desde [2,0]):
[[2, 0], [2, 1], [2, 2], [2, 3], [1, 3], [0, 3]]

==================== SNAPSHOT 0 ====================


,up,down,left,right
Position,,,,
"(0, 0)",0.0,0.0,0.0,0.0
"(0, 1)",0.0,0.0,0.0,0.0
"(0, 2)",0.0,0.0,0.0,0.0
"(0, 3)",0.0,0.0,0.0,0.0
"(1, 0)",0.0,0.0,0.0,0.0
"(1, 1)",0.0,0.0,0.0,0.0
"(1, 2)",0.0,0.0,0.0,0.0
"(1, 3)",0.0,0.0,0.0,0.0
"(2, 0)",0.0,0.0,0.0,0.0



==================== SNAPSHOT 1 ====================


,up,down,left,right
Position,,,,
"(0, 0)",13.008685,0.188933,7.482449,60.594546
"(0, 1)",17.386681,21.044826,9.546602,82.485847
"(0, 2)",32.175467,20.193297,24.769101,97.496844
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",33.594945,9.091894,10.476302,4.425725
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",52.017925,26.210108,48.401013,93.444415
"(1, 3)",99.999961,75.537967,75.927125,87.246569
"(2, 0)",15.153372,60.585044,64.366612,77.648588



==================== SNAPSHOT 2 ====================


,up,down,left,right
Position,,,,
"(0, 0)",13.008685,0.188933,7.482449,82.265689
"(0, 1)",17.386681,27.329545,20.628823,92.013315
"(0, 2)",32.175467,43.526415,24.769101,99.624290
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",68.012034,15.467244,19.180453,7.074672
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",56.002113,36.978254,55.919140,93.955681
"(1, 3)",100.000000,85.672385,85.381202,90.769852
"(2, 0)",50.401541,69.730852,69.193864,77.740750



==================== SNAPSHOT 3 ====================


,up,down,left,right
Position,,,,
"(0, 0)",13.008685,0.188933,7.482449,84.391073
"(0, 1)",24.332185,33.304021,20.628823,92.817738
"(0, 2)",38.334503,43.526415,24.769101,99.800332
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",72.768719,15.467244,23.623551,7.074672
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",56.002113,36.978254,55.919140,93.988735
"(1, 3)",100.000000,87.043220,86.566449,91.880700
"(2, 0)",57.644470,70.804804,70.452486,77.740750


In [83]:
# ==========================================
# Ejecución Ejercicio 1.b (Entorno variado)
# ==========================================

print("=== Configurando Entorno 1.b ===")

# Definición del entorno 1.b
rewards_b = np.array([
    [-3., -2., -1., GOAL_SCORE],
    [-4., -100., -2., -1.],  # Zona de peligro (le ponemos -100 pero en realidad se evita con obstacles)
    [-5., -4., -3., -2.]
])
rewards_b[0, 3] = GOAL_SCORE # Asegurar meta
obstacles_b = [[1, 1]]       # Mismo obstáculo

print("Recompensas 1.b:")
print(rewards_b)

# 1. Reiniciamos Q-Table para 1.b
# Usamos grid_rows y grid_cols (minúsculas)
q_table_b = np.zeros((grid_rows, grid_cols, 4))

# 2. Entrenamos
print("\nEntrenando Agente 1.b...")
results_1b = train(
    q_table=q_table_b, 
    grid_rewards=rewards_b, 
    obstacles=obstacles_b, 
    alpha=ALPHA,
    gamma=GAMMA,
    epsilon=EPSILON,
    conv_threshold=0.0001, 
    patience=20,
    drunkenness=0.0
)

# 3. Resultados
print(f"\nEntrenamiento 1.b finalizado en {results_1b['total_epochs']} épocas.")
print(f"¿Convergió?: {results_1b['is_converged']}")
print(f"Recompensa Promedio Final: {results_1b['avg_reward_final']:.2f}")

# 4. Verificación del Camino Aprendido
print("\nCamino Óptimo Aprendido 1.b (desde [2,0]):")
path_1b = get_shortest_path(2, 0, q_table_b, rewards_b, obstacles_b)
print(path_1b)

# 5. Visualización de las Q-Tables
snapshots_1b = results_1b['snapshots']
display_q_table_snapshots(snapshots_1b)

=== Configurando Entorno 1.b ===
Recompensas 1.b:
[[  -3.   -2.   -1.  100.]
 [  -4. -100.   -2.   -1.]
 [  -5.   -4.   -3.   -2.]]

Entrenando Agente 1.b...
Iniciando entrenamiento...


  2%|▏         | 663/30000 [00:00<00:05, 5425.79it/s]

Converged after 664 epochs.

Entrenamiento 1.b finalizado en 664 épocas.
¿Convergió?: True
Recompensa Promedio Final: 89.44

Camino Óptimo Aprendido 1.b (desde [2,0]):
[[2, 0], [1, 0], [0, 0], [0, 1], [0, 2], [0, 3]]

==================== SNAPSHOT 0 ====================


,up,down,left,right
Position,,,,
"(0, 0)",0.0,0.0,0.0,0.0
"(0, 1)",0.0,0.0,0.0,0.0
"(0, 2)",0.0,0.0,0.0,0.0
"(0, 3)",0.0,0.0,0.0,0.0
"(1, 0)",0.0,0.0,0.0,0.0
"(1, 1)",0.0,0.0,0.0,0.0
"(1, 2)",0.0,0.0,0.0,0.0
"(1, 3)",0.0,0.0,0.0,0.0
"(2, 0)",0.0,0.0,0.0,0.0



==================== SNAPSHOT 1 ====================


,up,down,left,right
Position,,,,
"(0, 0)",75.892236,67.767223,75.574711,87.299878
"(0, 1)",85.053019,82.809425,76.860581,93.999980
"(0, 2)",89.048010,68.168042,72.434003,99.999997
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.934604,56.810255,67.087242,65.047951
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",50.688333,12.309954,38.968417,89.150217
"(1, 3)",99.726107,48.659594,47.094198,58.144920
"(2, 0)",71.936998,53.929806,56.481721,26.993150



==================== SNAPSHOT 2 ====================


,up,down,left,right
Position,,,,
"(0, 0)",78.194721,69.943260,77.849482,87.300000
"(0, 1)",86.225276,85.890809,78.611563,94.000000
"(0, 2)",92.273348,82.324553,80.900674,100.000000
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.935000,60.810973,70.705175,69.268804
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",58.917549,12.309954,47.861663,93.274135
"(1, 3)",99.978153,48.659594,57.405474,58.144920
"(2, 0)",71.938250,59.289831,60.388441,43.903532



==================== SNAPSHOT 3 ====================


,up,down,left,right
Position,,,,
"(0, 0)",78.525374,70.483902,78.566692,87.300000
"(0, 1)",86.594873,86.375430,78.863016,94.000000
"(0, 2)",92.867143,84.433635,83.101402,100.000000
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.935000,61.681165,70.939459,69.992224
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",58.917549,12.309954,47.861663,93.645651
"(1, 3)",99.990595,48.659594,60.325969,58.144920
"(2, 0)",71.938250,59.694982,60.949491,46.238516


**Drunken sailor**

In [86]:
# ==========================================
# Ejecución Ejercicio 1.c (Entorno variado despues de 10L de sangria)
# ==========================================

print("=== Entrenando Drunken Sailor (1.c) ===")

drunkenness_level = 0.01 # 1% de probabilidad de resbalar/acción aleatoria (99% de acción correcta)

# 1. Configuración
q_table_drunk = np.zeros((grid_rows, grid_cols, 4))

# 2. Entrenamiento
results_drunk = train(
    q_table_drunk, 
    rewards_b, 
    obstacles_b, 
    conv_threshold=0.0001, 
    patience=20, 
    drunkenness=drunkenness_level
)

# 3. Resultados Individuales
print(f"\nEntrenamiento (Drunkenness={drunkenness_level}) finalizado en {results_drunk['total_epochs']} épocas.")
print(f"¿Convergió?: {results_drunk['is_converged']}")
print(f"Recompensa Final: {results_drunk['avg_reward_final']:.2f}")

# 4. Verificación del Camino
print("\nCamino aprendido por el agente borracho:")
path_drunk = get_shortest_path(2, 0, q_table_drunk, rewards_b, obstacles_b)
print(path_drunk)

# 5. Visualización de las Q-Tables
snapshots_drunk = results_drunk['snapshots']
display_q_table_snapshots(snapshots_drunk)

=== Entrenando Drunken Sailor (1.c) ===
Iniciando entrenamiento (Drunkenness=0.01)...


  2%|▏         | 661/30000 [00:00<00:06, 4251.18it/s]

Converged after 662 epochs.

Entrenamiento (Drunkenness=0.01) finalizado en 662 épocas.
¿Convergió?: True
Recompensa Final: 89.25

Camino aprendido por el agente borracho:
[[2, 0], [1, 0], [0, 0], [0, 1], [0, 2], [0, 3]]

==================== SNAPSHOT 0 ====================


,up,down,left,right
Position,,,,
"(0, 0)",0.0,0.0,0.0,0.0
"(0, 1)",0.0,0.0,0.0,0.0
"(0, 2)",0.0,0.0,0.0,0.0
"(0, 3)",0.0,0.0,0.0,0.0
"(1, 0)",0.0,0.0,0.0,0.0
"(1, 1)",0.0,0.0,0.0,0.0
"(1, 2)",0.0,0.0,0.0,0.0
"(1, 3)",0.0,0.0,0.0,0.0
"(2, 0)",0.0,0.0,0.0,0.0



==================== SNAPSHOT 1 ====================


,up,down,left,right
Position,,,,
"(0, 0)",77.972734,69.894070,78.602733,87.299988
"(0, 1)",84.123604,83.511702,78.200865,93.999998
"(0, 2)",92.915148,80.136224,84.745473,100.000000
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.934946,58.621217,67.215159,68.656149
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",93.527637,28.379257,61.345974,57.233569
"(1, 3)",89.058101,2.014973,11.326754,31.451837
"(2, 0)",71.938006,60.210967,58.070638,41.550263



==================== SNAPSHOT 2 ====================


,up,down,left,right
Position,,,,
"(0, 0)",79.250800,71.559457,79.558728,87.300000
"(0, 1)",86.069399,86.433359,79.390810,94.000000
"(0, 2)",93.724245,84.681663,86.578526,100.000000
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.935000,62.466684,70.965806,70.793850
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",93.936191,41.701560,63.911934,59.870731
"(1, 3)",90.152291,2.014973,11.326754,31.451837
"(2, 0)",71.938250,62.128555,62.001577,55.623209



==================== SNAPSHOT 3 ====================


,up,down,left,right
Position,,,,
"(0, 0)",79.319220,71.714577,79.596355,87.300000
"(0, 1)",86.402892,86.598021,79.577957,94.000000
"(0, 2)",93.776639,86.367465,86.873977,100.000000
"(0, 3)",0.000000,0.000000,0.000000,0.000000
"(1, 0)",79.935000,62.554149,71.473133,71.330069
"(1, 1)",0.000000,0.000000,0.000000,0.000000
"(1, 2)",93.985402,41.701560,63.911934,59.870731
"(1, 3)",90.152291,2.014973,11.326754,31.451837
"(2, 0)",71.938250,62.457219,62.364652,59.516486


In [ ]:
# ==========================================
# Comparación: Impacto del "Drunkenness"
# ==========================================

runs_comparison = 10 
epochs_sober_list = []
epochs_drunk_list = []

print(f"Iniciando comparativa ({runs_comparison} runs por configuración)...")

t = trange(runs_comparison, desc="Runs")
for _ in t:
    # A. Normal/sobrio (Drunkenness = 0.0)
    qt_sober = np.zeros((grid_rows, grid_cols, 4))
    res_s = train(
        qt_sober, rewards_b, obstacles_b,
        alpha=0.1, gamma=0.99, epsilon=0.8,
        conv_threshold=0.001, patience=20, drunkenness=0.0
    )
    if res_s['is_converged']:
        epochs_sober_list.append(res_s['total_epochs'])
        
    # B. Borracho (Drunkenness = 0.01)
    qt_drunk = np.zeros((grid_rows, grid_cols, 4))
    res_d = train(
        qt_drunk, rewards_b, obstacles_b, 
        alpha=0.1, gamma=0.99, epsilon=0.8,
        conv_threshold=0.001, patience=20, drunkenness=0.01
    )
    if res_d['is_converged']:
        epochs_drunk_list.append(res_d['total_epochs'])

# Comparativa 
mean_sober = np.mean(epochs_sober_list)
mean_drunk = np.mean(epochs_drunk_list)
impact_pct = ((mean_drunk - mean_sober) / mean_sober) * 100

print("\n" + "="*40)
print(f"RESULTADOS (Promedio de {runs_comparison} runs)")
print("="*40)
print(f"Sobrio (0.0):   {mean_sober:.1f} epochs")
print(f"Borracho (0.01): {mean_drunk:.1f} epochs")
print("-" * 40)
print(f"Impacto de la borrachera: +{impact_pct:.1f}% de tiempo para converger")

Iniciando comparativa (10 runs por configuración)...


Runs:   0%|          | 0/10 [00:00<?, ?it/s]

Iniciando entrenamiento...


  2%|▏         | 738/30000 [00:00<00:05, 5293.21it/s]


Converged after 739 epochs.
Iniciando entrenamiento (Drunkenness=0.01)...


...

Converged after 682 epochs.
Iniciando entrenamiento...


  2%|▏         | 656/30000 [00:00<00:05, 5069.99it/s]


Converged after 657 epochs.
Iniciando entrenamiento (Drunkenness=0.01)...


Runs: 100%|██████████| 10/10 [00:03<00:00,  3.31it/s]

Converged after 796 epochs.

RESULTADOS (Promedio de 10 runs)
Sobrio (0.0):   662.1 epochs
Borracho (0.1): 711.8 epochs
----------------------------------------
Impacto del Ruido: +7.5% de tiempo para converger
